In [1]:
!pip install optimum[openvino]
!pip install gradio diffusers optimum torchaudio moviepy requests serpapi
!pip install google-search-results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import gradio as gr
import torch
import torchaudio
import os
import warnings
from moviepy.editor import VideoFileClip
from transformers import pipeline, AutoProcessor, AutoTokenizer, AutoModelForSeq2SeqLM
from optimum.intel.openvino import OVModelForSpeechSeq2Seq, OVModelForSeq2SeqLM
from diffusers import StableDiffusionXLPipeline, StableDiffusionPipeline
from io import BytesIO
from serpapi import GoogleSearch
import numpy as np
import gc
import time

  if event.key is 'enter':



In [3]:
warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
UPLOAD_FOLDER = "uploads"
MODEL_CACHE = "model_cache"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(MODEL_CACHE, exist_ok=True)

In [5]:
print("🔄 Initializing Whisper Small model...")
whisper_model_path = os.path.join(MODEL_CACHE, "whisper-small-ov")
if not os.path.exists(whisper_model_path):
    print("🔄 Loading Whisper Small model and converting to OpenVINO...")
    whisper_model = OVModelForSpeechSeq2Seq.from_pretrained("openai/whisper-small", export=True)
    whisper_model.save_pretrained(whisper_model_path)
    print("✅ Whisper Small OpenVINO model saved!")
else:
    print("📂 Loading existing Whisper Small OpenVINO model...")
    whisper_model = OVModelForSpeechSeq2Seq.from_pretrained(whisper_model_path)

whisper_processor = AutoProcessor.from_pretrained("openai/whisper-small")

🔄 Initializing Whisper Small model...
🔄 Loading Whisper Small model and converting to OpenVINO...


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning becau

✅ Whisper Small OpenVINO model saved!


In [6]:
print("🔄 Initializing T5 Summarizer...")
summarizer_model_path = os.path.join(MODEL_CACHE, "t5-base-summarizer-ov")
if not os.path.exists(summarizer_model_path):
    print("🔄 Loading T5 summarizer and converting to OpenVINO...")
    summarizer_model = OVModelForSeq2SeqLM.from_pretrained("t5-base", export=True)
    summarizer_model.save_pretrained(summarizer_model_path)
    print("✅ T5 Summarizer OpenVINO model saved!")
else:
    print("📂 Loading existing T5 Summarizer OpenVINO model...")
    summarizer_model = OVModelForSeq2SeqLM.from_pretrained(summarizer_model_path)

summarizer_tokenizer = AutoTokenizer.from_pretrained("t5-base")


🔄 Initializing T5 Summarizer...
🔄 Loading T5 summarizer and converting to OpenVINO...


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

BetterTransformer requires transformers<4.49 but found 4.51.3. `optimum.bettertransformer` is deprecated and will be removed in optimum v2.0.. Usage model with stateful=True may be non-effective if model does not contain torch.functional.scaled_dot_product_attention
  or not self.key_cache[layer_idx].numel()  # the layer has no cache

  if sequence_length != 1:

  elif (



✅ T5 Summarizer OpenVINO model saved!


In [7]:
print("🔄 Setting up Stable Diffusion (lightweight mode)...")
stable_pipe = None  # Will load on first use

def load_stable_diffusion():
    """Load SD model only when needed"""
    global stable_pipe
    if stable_pipe is not None:
        return stable_pipe

    try:
        print("🔄 Loading Stable Diffusion v1.5 (on-demand)...")
        stable_pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            low_cpu_mem_usage=True,
            use_safetensors=True
        )

        if torch.cuda.is_available():
            stable_pipe = stable_pipe.to("cuda")
        else:
            stable_pipe = stable_pipe.to("cpu")

        print("✅ Stable Diffusion loaded successfully!")
        return stable_pipe

    except Exception as e:
        print(f"❌ Stable Diffusion loading failed: {e}")
        return None

🔄 Setting up Stable Diffusion (lightweight mode)...


In [8]:
def transcribe_audio_from_video(video_file, progress=gr.Progress()):
    if not video_file:
        return "❌ No video uploaded."

    try:
        progress(0, desc="Extracting audio...")

        video = VideoFileClip(video_file)
        if not video.audio:
            return "❌ No audio found in video."

        audio_path = os.path.join(UPLOAD_FOLDER, "audio.wav")
        video.audio.write_audiofile(audio_path, logger=None)
        video.close()

        waveform, sample_rate = torchaudio.load(audio_path)

        if sample_rate != 16000:
            resample = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            waveform = resample(waveform)
            sample_rate = 16000

        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        progress(0.2, desc="Processing audio chunks...")

        chunk_length_sec = 25
        chunk_size = chunk_length_sec * sample_rate
        stride = int(sample_rate * 5)
        transcripts = []

        total_chunks = max(1, (waveform.size(1) - chunk_size) // stride + 1)

        for i, start in enumerate(range(0, waveform.size(1), chunk_size - stride)):
            end = min(start + chunk_size, waveform.size(1))
            chunk = waveform[:, start:end]

            if chunk.size(1) < 16000:
                continue

            try:
                inputs = whisper_processor(chunk.squeeze(0), sampling_rate=16000, return_tensors="pt")
                generated_ids = whisper_model.generate(inputs["input_features"])
                text = whisper_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

                if text and text.strip():
                    transcripts.append(text.strip())

                progress((i + 1) / total_chunks * 0.8 + 0.2, desc=f"Chunk {i+1}/{total_chunks}")

            except Exception as e:
                print(f"❌ Chunk {i} failed: {e}")
                continue

        if os.path.exists(audio_path):
            os.remove(audio_path)

        progress(1.0, desc="Transcription complete!")

        if not transcripts:
            return "❌ No speech detected in video. Try a video with clearer audio."

        final_transcript = " ".join(transcripts)
        return final_transcript if final_transcript.strip() else "❌ No clear speech detected."

    except Exception as e:
        return f"❌ Error processing video: {str(e)}"


In [9]:
def summarize_text_enhanced(text, min_length=60, max_length=600):
    if not text.strip():
        return "❌ No text provided."

    try:
        input_text = f"summarize: {text}"
        inputs = summarizer_tokenizer(
            input_text,
            max_length=1024,
            truncation=True,
            return_tensors="pt"
        )

        summary_ids = summarizer_model.generate(
            inputs["input_ids"],
            max_new_tokens=max_length,
            min_length=min_length,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3,
            do_sample=False
        )

        summary = summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        del inputs, summary_ids
        gc.collect()

        return summary

    except Exception as e:
        return f"❌ Enhanced summarization error: {str(e)}"


In [10]:
def generate_image_enhanced(prompt, negative_prompt="blurry, low quality, distorted", steps=20):
    if not prompt.strip():
        return None

    pipe = load_stable_diffusion()
    if not pipe:
        return None

    try:
        enhanced_prompt = f"high quality, detailed, {prompt}"

        image = pipe(
            prompt=enhanced_prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=steps,
            guidance_scale=7.5,
            width=512,
            height=512,
            generator=torch.Generator().manual_seed(42)
        ).images[0]

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        return image

    except Exception as e:
        print(f"❌ Image generation failed: {e}")
        return None

In [11]:
def search_web_enhanced(query, num_results=8):
    if not query.strip():
        return "❌ No search query provided."

    try:
        search = GoogleSearch({
            "q": query,
            "api_key": "cbc9aa754b28b7a45c57feb677147a418d633f661ef678900c64e24bc52c379a"
        })

        results = search.get_dict()

        if "error" in results:
            return f"❌ Search API error: {results['error']}"

        organic_results = results.get("organic_results", [])

        if not organic_results:
            return "❌ No search results found for this query."

        formatted_results = []
        for i, result in enumerate(organic_results[:num_results], 1):
            title = result.get("title", "No title")
            link = result.get("link", "")
            snippet = result.get("snippet", "No description available")

            result_text = f"{i}. {title}\n{snippet}\nLink: {link}\n"
            formatted_results.append(result_text)

        return "\n".join(formatted_results)

    except Exception as e:
        return f"❌ Search error: {str(e)}\nPlease check your API key and internet connection."

In [12]:
def extract_keywords_and_concepts(text, max_keywords=10):
    """Extract key concepts and topics from text for better correlation"""
    import re

    # Common educational/academic keywords that should be prioritized
    priority_terms = [
        'theory', 'concept', 'principle', 'method', 'process', 'system', 'model',
        'analysis', 'research', 'study', 'experiment', 'data', 'result', 'conclusion',
        'algorithm', 'function', 'equation', 'formula', 'definition', 'example',
        'application', 'implementation', 'solution', 'problem', 'approach'
    ]

    # Clean and tokenize text
    words = re.findall(r'\b[a-zA-Z]{4,}\b', text.lower())

    # Count word frequency
    word_freq = {}
    for word in words:
        if len(word) >= 4:  # Only consider meaningful words
            word_freq[word] = word_freq.get(word, 0) + 1

    # Boost priority terms
    for word in word_freq:
        if any(term in word for term in priority_terms):
            word_freq[word] *= 2

    # Get top keywords
    top_keywords = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:max_keywords]
    return [word for word, freq in top_keywords]

In [13]:
def create_focused_search_queries(summary, transcript, num_queries=3):
    """Create multiple focused search queries from content"""
    keywords = extract_keywords_and_concepts(summary + " " + transcript[:500])

    # Extract main topics from first few sentences
    sentences = [s.strip() for s in summary.split('.') if len(s.strip()) > 20]

    queries = []

    if len(sentences) >= 1:
        # Main topic query
        main_keywords = keywords[:3]
        queries.append(" ".join(main_keywords) + " tutorial explanation")

    if len(sentences) >= 2:
        # Secondary topic query
        secondary_keywords = keywords[2:5] if len(keywords) > 2 else keywords
        queries.append(" ".join(secondary_keywords) + " examples")

    # Academic/educational resource query
    if len(keywords) >= 3:
        academic_keywords = keywords[:2]
        queries.append(" ".join(academic_keywords) + " research paper PDF")

    return queries[:num_queries]

In [14]:
def create_contextual_image_prompts(summary, transcript):
    """Create visual-focused image prompts without text elements"""
    keywords = extract_keywords_and_concepts(summary + " " + transcript[:300])
    sentences = [s.strip() for s in summary.split('.') if len(s.strip()) > 15]

    prompts = []

    # Analyze content type and create appropriate visual prompts (NO TEXT)
    content_lower = (summary + " " + transcript[:200]).lower()

    # Scientific/Technical content - Focus on visual representations
    if any(term in content_lower for term in ['algorithm', 'data', 'analysis', 'research', 'experiment', 'scientific']):
        if keywords:
            prompts.append(f"abstract scientific visualization of {' '.join(keywords[:3])}, molecular structures, geometric patterns, blue and white color scheme, no text, no labels")
            prompts.append(f"laboratory equipment and scientific instruments related to {' '.join(keywords[1:4])}, clean photography style, professional lighting, no text")

    # Mathematical content - Focus on geometric and abstract visuals
    elif any(term in content_lower for term in ['equation', 'formula', 'mathematical', 'calculation', 'function']):
        if keywords:
            prompts.append(f"abstract geometric patterns representing {' '.join(keywords[:3])}, mathematical beauty, golden ratio, fractals, colorful visualization, no text")
            prompts.append(f"3D mathematical shapes and forms illustrating {' '.join(keywords[1:4])}, clean render, educational sculpture, no text")

    # Historical/Social content - Focus on scenes and imagery
    elif any(term in content_lower for term in ['history', 'historical', 'society', 'culture', 'people', 'event']):
        if keywords:
            prompts.append(f"historical scene depicting {' '.join(keywords[:3])}, period accurate clothing and architecture, oil painting style, no text")
            prompts.append(f"cultural artifacts and symbols related to {' '.join(keywords[1:4])}, museum display style, artistic photography, no text")

    # Business/Economic content - Focus on visual metaphors
    elif any(term in content_lower for term in ['business', 'economic', 'market', 'financial', 'company']):
        if keywords:
            prompts.append(f"business concept visualization of {' '.join(keywords[:3])}, modern office environment, professional photography, no text")
            prompts.append(f"abstract representation of {' '.join(keywords[1:4])}, growth metaphors, upward trending visuals, clean corporate style, no text")

    # Technology/Computer Science content
    elif any(term in content_lower for term in ['technology', 'computer', 'software', 'programming', 'digital']):
        if keywords:
            prompts.append(f"futuristic technology visualization of {' '.join(keywords[:3])}, circuit board patterns, neon colors, digital art style, no text")
            prompts.append(f"abstract network connections representing {' '.join(keywords[1:4])}, nodes and connections, cyberpunk aesthetic, no text")

    # Biology/Medical content
    elif any(term in content_lower for term in ['biology', 'medical', 'health', 'anatomy', 'cell', 'organism']):
        if keywords:
            prompts.append(f"biological visualization of {' '.join(keywords[:3])}, organic forms, cell structures, microscopic beauty, scientific photography, no text")
            prompts.append(f"anatomical illustration style showing {' '.join(keywords[1:4])}, medical art, detailed drawings, educational poster style, no text")

    # General educational content - Focus on concepts and metaphors
    else:
        if len(sentences) >= 1 and keywords:
            prompts.append(f"conceptual art representing {' '.join(keywords[:3])}, abstract visualization, educational concept, colorful illustration, no text, no labels")

        if len(keywords) >= 3:
            prompts.append(f"symbolic representation of {' '.join(keywords[:3])}, metaphorical imagery, artistic interpretation, clean design, no text")

    # Fallback prompts if none generated
    if not prompts and keywords:
        prompts.extend([
            f"abstract artistic representation of {' '.join(keywords[:2])}, conceptual art, educational theme, no text",
            f"visual metaphor for {' '.join(keywords[2:4]) if len(keywords) > 2 else keywords[0]}, symbolic imagery, clean artistic style, no text"
        ])

    return prompts[:2]  # Return max 2 prompts

In [15]:
def generate_image_enhanced(prompt, negative_prompt="blurry, low quality, distorted", steps=20):
    """Enhanced image generation with strong anti-text measures"""
    if not prompt.strip():
        return None

    pipe = load_stable_diffusion()
    if not pipe:
        return None

    try:
        # Enhanced prompt for better visual quality
        enhanced_prompt = f"high quality, detailed, professional, {prompt}"

        # Strong negative prompt to prevent text generation
        comprehensive_negative_prompt = f"{negative_prompt}, text, letters, words, writing, labels, captions, titles, watermark, signature, typography, alphabet, numbers, symbols, character, font, script, handwriting, printed text, readable text, any text elements"

        image = pipe(
            prompt=enhanced_prompt,
            negative_prompt=comprehensive_negative_prompt,
            num_inference_steps=steps,
            guidance_scale=7.5,
            width=512,
            height=512,
            generator=torch.Generator().manual_seed(42)
        ).images[0]

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        return image

    except Exception as e:
        print(f"❌ Image generation failed: {e}")
        return None

In [16]:
def process_video_lecture_pipeline(video_file, min_summary_words=100, max_summary_words=500,
                                 num_search_results=6, num_images=2, progress=gr.Progress()):
    """
    Enhanced pipeline with better correlation: Video → Transcript → Summary → Contextual Images + Focused Search
    """
    if not video_file:
        return "❌ No video uploaded.", "", "", None, None, ""

    try:
        # Step 1: Transcription (20% of progress)
        progress(0, desc="🎬 Starting video transcription...")
        transcript = transcribe_audio_from_video(video_file, progress=lambda p, desc: progress(p * 0.2, desc))

        if transcript.startswith("❌"):
            return transcript, "", "", None, None, ""

        progress(0.2, desc="✅ Transcription complete!")

        # Step 2: Summarization (40% of progress)
        progress(0.25, desc="📋 Generating summary...")
        summary = summarize_text_enhanced(transcript, min_summary_words, max_summary_words)

        if summary.startswith("❌"):
            return transcript, summary, "", None, None, ""

        progress(0.4, desc="✅ Summary generated!")

        # Step 3: Enhanced content analysis and topic extraction
        progress(0.45, desc="🔍 Analyzing content and extracting key concepts...")

        # Extract keywords and create focused queries
        keywords = extract_keywords_and_concepts(summary + " " + transcript[:500])
        search_queries = create_focused_search_queries(summary, transcript, 2)

        progress(0.5, desc="🌐 Performing focused web searches...")

        # Step 4: Enhanced Web Search with multiple focused queries
        all_search_results = []
        for i, query in enumerate(search_queries):
            progress(0.5 + (i * 0.05), desc=f"🔍 Searching: {query[:30]}...")
            results = search_web_enhanced(query, num_search_results // len(search_queries) + 1)
            if not results.startswith("❌"):
                all_search_results.append(f"📋 **Query: {query}**\n{results}\n")

        # Combine search results
        final_search_results = "\n".join(all_search_results) if all_search_results else "❌ No search results found"

        progress(0.6, desc="✅ Search complete!")

        # Step 5: Enhanced contextual image generation (NO TEXT FOCUS)
        progress(0.65, desc="🎨 Creating visual concept prompts...")

        # Generate visual-focused prompts based on content analysis
        contextual_prompts = create_contextual_image_prompts(summary, transcript)

        generated_images = []
        for i, prompt in enumerate(contextual_prompts[:num_images]):
            progress(0.65 + (i * 0.15), desc=f"🎨 Generating visual concept {i+1}/{num_images}...")
            image = generate_image_enhanced(
                prompt,
                negative_prompt="blurry, low quality, distorted, text, letters, words, writing, labels, captions, titles, watermark, signature, typography, alphabet, numbers, symbols, readable text",
                steps=20  # Higher quality for better visual concepts
            )
            generated_images.append(image)

        progress(0.95, desc="🎯 Finalizing enhanced results...")

        # Prepare results
        image1 = generated_images[0] if len(generated_images) > 0 else None
        image2 = generated_images[1] if len(generated_images) > 1 else None

        # Create enhanced result summary
        pipeline_summary = f"""
🎓 **ENHANCED CLASSROOM ASSISTANT - LECTURE ANALYSIS COMPLETE**

📊 **Processing Summary:**
• Video processed successfully ✅
• Transcript: {len(transcript.split())} words | Summary: {len(summary.split())} words
• Key concepts identified: {', '.join(keywords[:5])}
• Generated {len([img for img in generated_images if img is not None])} visual concept images (text-free)
• Performed {len(search_queries)} focused searches with {num_search_results} total results

🔍 **Search Strategies Used:**
{chr(10).join([f"• {query}" for query in search_queries])}

🎨 **Visual Concept Generation:**
{chr(10).join([f"• {prompt[:80]}..." for prompt in contextual_prompts[:2]])}

📈 **Content Analysis:**
• Primary topics: {', '.join(keywords[:3])}
• Secondary concepts: {', '.join(keywords[3:6]) if len(keywords) > 3 else 'N/A'}
        """.strip()

        progress(1.0, desc="✅ Enhanced processing complete! All components correlated successfully.")

        return transcript, summary, pipeline_summary, image1, image2, final_search_results

    except Exception as e:
        error_msg = f"❌ Enhanced pipeline error: {str(e)}"
        return error_msg, "", "", None, None, ""

In [19]:
custom_css = """
.gradio-container {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background: linear-gradient(135deg, #ffffff 0%, #e3f2fd 50%, #bbdefb 100%);
    color: #000000 !important;
}

/* Force all text to be black and backgrounds to be white/light */
* {
    color: #000000 !important;
}

/* Fix all text input and output areas */
.gr-textbox,
.gr-textbox textarea,
.gr-textbox input,
textarea,
input[type="text"],
.gr-form > div > div textarea,
.gr-form textarea {
    background: #ffffff !important;
    color: #000000 !important;
    border: 2px solid #bbdefb !important;
    border-radius: 8px !important;
}

.gr-textbox:focus,
.gr-textbox textarea:focus,
.gr-textbox input:focus,
textarea:focus,
input[type="text"]:focus {
    border-color: #2196f3 !important;
    box-shadow: 0 0 8px rgba(33, 150, 243, 0.3) !important;
    background: #ffffff !important;
    color: #000000 !important;
}

/* Fix file upload areas */
.gr-file {
    background: #ffffff !important;
    color: #000000 !important;
    border: 2px solid #bbdefb !important;
    border-radius: 8px !important;
}

/* Fix video display */
.gr-video {
    background: #ffffff !important;
    border: 2px solid #bbdefb !important;
    border-radius: 8px !important;
}

/* Fix image display containers */
.gr-image {
    background: #ffffff !important;
    border: 2px solid #bbdefb !important;
    border-radius: 8px !important;
}

/* Fix all form containers and panels */
.gr-panel,
.gr-box,
.gr-form,
div[data-testid="block-container"],
.gr-column,
.gr-row {
    background: transparent !important;
    color: #000000 !important;
}

/* Fix specific problematic containers */
div.gr-group,
div.gr-padded,
.gradio-accordion,
.gr-accordion {
    background: #ffffff !important;
    color: #000000 !important;
    border: 1px solid #bbdefb !important;
    border-radius: 8px !important;
    padding: 15px !important;
}

/* Fix labels */
.gr-label,
label,
.gr-textbox label,
.gr-file label,
.gr-image label,
.gr-video label,
.gr-slider label {
    color: #000000 !important;
    font-weight: 600 !important;
    background: transparent !important;
}

/* Tab navigation styling */
.tab-nav button {
    font-size: 16px;
    font-weight: 600;
    padding: 12px 24px;
    background: linear-gradient(135deg, #2196f3 0%, #1976d2 100%);
    color: white !important;
    border: none;
    border-radius: 10px;
    margin: 2px;
    box-shadow: 0 3px 6px rgba(33, 150, 243, 0.3);
    transition: all 0.3s ease;
}

.tab-nav button:hover {
    background: linear-gradient(135deg, #1976d2 0%, #0d47a1 100%);
    transform: translateY(-2px);
    box-shadow: 0 5px 10px rgba(33, 150, 243, 0.4);
    color: white !important;
}

.tab-nav button.selected {
    background: linear-gradient(135deg, #0d47a1 0%, #01579b 100%);
    color: white !important;
}

/* Header styling */
.main-header {
    text-align: center;
    background: linear-gradient(135deg, #1976d2 0%, #0d47a1 100%);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    font-size: 2.5em;
    font-weight: bold;
    margin-bottom: 20px;
}

/* Card styling */
.feature-card {
    border: 2px solid #2196f3;
    border-radius: 15px;
    padding: 20px;
    margin: 10px 0;
    background: #ffffff !important;
    box-shadow: 0 4px 8px rgba(33, 150, 243, 0.2);
    color: #000000 !important;
}

.pipeline-card {
    border: 3px solid #1976d2;
    border-radius: 20px;
    padding: 25px;
    margin: 15px 0;
    background: #ffffff !important;
    box-shadow: 0 6px 12px rgba(25, 118, 210, 0.3);
    color: #000000 !important;
}

/* Button styling */
.gr-button {
    background: linear-gradient(135deg, #2196f3 0%, #1976d2 100%) !important;
    color: white !important;
    border: none !important;
    border-radius: 12px !important;
    padding: 12px 24px !important;
    font-weight: 600 !important;
    font-size: 16px !important;
    box-shadow: 0 4px 8px rgba(33, 150, 243, 0.3) !important;
    transition: all 0.3s ease !important;
}

.gr-button:hover {
    background: linear-gradient(135deg, #1976d2 0%, #0d47a1 100%) !important;
    transform: translateY(-2px) !important;
    box-shadow: 0 6px 12px rgba(33, 150, 243, 0.4) !important;
    color: white !important;
}

.gr-button.primary {
    background: linear-gradient(135deg, #1976d2 0%, #0d47a1 100%) !important;
    color: white !important;
}

.gr-button.primary:hover {
    background: linear-gradient(135deg, #0d47a1 0%, #01579b 100%) !important;
    color: white !important;
}

/* Special pipeline button */
.pipeline-btn {
    background: linear-gradient(135deg, #4caf50 0%, #2e7d32 100%) !important;
    font-size: 18px !important;
    padding: 15px 30px !important;
    box-shadow: 0 5px 10px rgba(76, 175, 80, 0.4) !important;
    color: white !important;
}

.pipeline-btn:hover {
    background: linear-gradient(135deg, #2e7d32 0%, #1b5e20 100%) !important;
    transform: translateY(-3px) !important;
    box-shadow: 0 8px 15px rgba(76, 175, 80, 0.5) !important;
    color: white !important;
}

/* Slider styling */
.gr-slider {
    background: #ffffff !important;
    border: 2px solid #bbdefb !important;
    border-radius: 8px !important;
    padding: 10px !important;
}

.gr-slider input[type="range"] {
    background: #2196f3 !important;
}

.gr-slider .gr-slider-label {
    color: #000000 !important;
    background: #ffffff !important;
}

/* Progress bar */
.gr-progress {
    background: #ffffff !important;
    border: 1px solid #bbdefb !important;
    border-radius: 8px !important;
}

.gr-progress-bar {
    background: linear-gradient(135deg, #2196f3 0%, #1976d2 100%) !important;
}

/* Fix markdown content */
.gr-markdown,
.gr-markdown p,
.gr-markdown h1,
.gr-markdown h2,
.gr-markdown h3,
.gr-markdown h4,
.gr-markdown h5,
.gr-markdown h6,
.gr-markdown ul,
.gr-markdown ol,
.gr-markdown li {
    color: #000000 !important;
    background: #ffffff !important;
}

/* Fix any remaining dark backgrounds */
div[style*="background-color: rgb(31, 41, 55)"],
div[style*="background-color: #1f2937"],
div[style*="background: rgb(31, 41, 55)"],
div[style*="background: #1f2937"] {
    background: #ffffff !important;
    color: #000000 !important;
}

/* Force override any dark theme */
.dark,
[data-theme="dark"],
.gr-theme-dark {
    background: #ffffff !important;
    color: #000000 !important;
}

/* Credits styling */
.credits {
    text-align: center;
    font-size: 14px;
    color: #1976d2 !important;
    font-weight: 600;
    margin: 20px 0;
    padding: 10px;
    background: linear-gradient(135deg, #e3f2fd 0%, #bbdefb 100%) !important;
    border-radius: 10px;
    border: 1px solid #2196f3;
}

/* Additional fixes for specific Gradio components */
.gr-interface {
    background: #ffffff !important;
    color: #000000 !important;
}

.gr-block.gr-padded {
    background: #ffffff !important;
    color: #000000 !important;
}

/* Ensure copy buttons are visible */
.gr-copy-button {
    background: #2196f3 !important;
    color: white !important;
    border: none !important;
    border-radius: 4px !important;
}

.gr-copy-button:hover {
    background: #1976d2 !important;
}
"""

# Gradio Interface
with gr.Blocks(css=custom_css, title="🧠 Enhanced Multimodal AI Classroom Assistant") as demo:
    gr.HTML('<div class="main-header">🧠 Enhanced Multimodal AI Classroom Assistant</div>')
    gr.Markdown("""
    ### 🎯 Advanced AI-Powered Classroom Assistant
    **Features:** Whisper-Small (OpenVINO) • T5 Summarizer (OpenVINO) • Stable Diffusion • Enhanced Search
    """)

    # NEW: Unified Pipeline Tab (Primary Feature)
    with gr.Tab("🎓 Complete Lecture Pipeline", elem_classes="pipeline-card"):
        gr.Markdown("""
        ### 🚀 **One-Click Lecture Processing**
        **Upload a video lecture and get everything automatically:**
        • 📝 Full transcript • 📋 Smart summary • 🎨 Educational images • 🔍 Related resources
        """)

        with gr.Row():
            with gr.Column(scale=1):
                pipeline_video_input = gr.Video(
                    label="🎬 Upload Video Lecture",
                    height=250
                )

                with gr.Accordion("⚙️ Advanced Settings", open=False):
                    pipeline_min_words = gr.Slider(30, 100, value=60, label="Min Summary Words")
                    pipeline_max_words = gr.Slider(200, 600, value=400, label="Max Summary Words")
                    pipeline_search_results = gr.Slider(3, 10, value=6, label="Search Results")
                    pipeline_num_images = gr.Slider(1, 3, value=2, label="Generated Images")

                pipeline_btn = gr.Button(
                    "🚀 Process Complete Lecture",
                    variant="primary",
                    size="lg",
                    elem_classes="pipeline-btn"
                )

            with gr.Column(scale=2):
                pipeline_status = gr.Textbox(
                    label="📊 Pipeline Status",
                    lines=8,
                    placeholder="Processing status will appear here...",
                    show_copy_button=True
                )

        # Results Section
        gr.Markdown("### 📋 **Processing Results**")

        with gr.Row():
            with gr.Column():
                pipeline_transcript = gr.Textbox(
                    label="📝 Generated Transcript",
                    lines=8,
                    placeholder="Full transcript will appear here...",
                    show_copy_button=True
                )

                pipeline_summary = gr.Textbox(
                    label="📋 Intelligent Summary",
                    lines=6,
                    placeholder="Summary will appear here...",
                    show_copy_button=True
                )

            with gr.Column():
                with gr.Row():
                    pipeline_image1 = gr.Image(label="🎨 Educational Image 1", height=200)
                    pipeline_image2 = gr.Image(label="🎨 Educational Image 2", height=200)

                pipeline_search = gr.Textbox(
                    label="🔍 Related Web Resources",
                    lines=8,
                    placeholder="Search results will appear here...",
                    show_copy_button=True
                )

        # Connect pipeline
        pipeline_btn.click(
            process_video_lecture_pipeline,
            inputs=[
                pipeline_video_input,
                pipeline_min_words,
                pipeline_max_words,
                pipeline_search_results,
                pipeline_num_images
            ],
            outputs=[
                pipeline_transcript,
                pipeline_summary,
                pipeline_status,
                pipeline_image1,
                pipeline_image2,
                pipeline_search
            ],
            show_progress=True
        )

    # Credits Section
    gr.HTML('<div class="credits">Made by Kewal Thacker and Siddharth Subramanian</div>')

    # Individual Component Tabs (for testing/debugging)
    with gr.Tab("🎬 Video Transcription", elem_classes="feature-card"):
        gr.Markdown("### 📝 Advanced Speech-to-Text with Whisper-Small (OpenVINO)")

        with gr.Row():
            with gr.Column(scale=2):
                video_input = gr.Video(label="📁 Upload Video File", height=300)
                trans_btn = gr.Button("🎯 Generate Transcript", variant="primary", size="lg")

            with gr.Column(scale=3):
                transcription_out = gr.Textbox(
                    label="📄 Generated Transcript",
                    lines=12,
                    placeholder="Transcript will appear here...",
                    show_copy_button=True
                )

        trans_btn.click(
            transcribe_audio_from_video,
            inputs=video_input,
            outputs=transcription_out,
            show_progress=True
        )

    with gr.Tab("📋 Text Summarization", elem_classes="feature-card"):
        gr.Markdown("### 🎯 Advanced T5-Based Summarization (OpenVINO)")

        with gr.Row():
            with gr.Column():
                summary_input = gr.Textbox(
                    label="📝 Input Text to Summarize",
                    lines=8,
                    placeholder="Paste your text here for summarization..."
                )

                with gr.Row():
                    min_words = gr.Slider(30, 100, value=60, label="Minimum Words")
                    max_words = gr.Slider(300, 800, value=600, label="Maximum Words")

                summary_btn = gr.Button("📊 Generate Summary", variant="primary", size="lg")

            with gr.Column():
                summary_out = gr.Textbox(
                    label="📄 Generated Summary",
                    lines=8,
                    placeholder="Summary will appear here...",
                    show_copy_button=True
                )

        summary_btn.click(
            lambda text, min_len, max_len: summarize_text_enhanced(text, min_len, max_len),
            inputs=[summary_input, min_words, max_words],
            outputs=summary_out
        )

    with gr.Tab("🎨 Image Generation", elem_classes="feature-card"):
        gr.Markdown("### 🖼️ Advanced Image Generation with Stable Diffusion")

        with gr.Row():
            with gr.Column():
                prompt_input = gr.Textbox(
                    label="✨ Image Prompt",
                    lines=3,
                    placeholder="Describe the image you want to generate..."
                )

                negative_prompt = gr.Textbox(
                    label="🚫 Negative Prompt (Optional)",
                    value="blurry, low quality, distorted, ugly",
                    lines=2
                )

                steps_slider = gr.Slider(10, 30, value=20, step=5, label="Quality Steps")
                image_btn = gr.Button("🎨 Generate Image", variant="primary", size="lg")

            with gr.Column():
                image_out = gr.Image(label="🖼️ Generated Image", height=400)

        image_btn.click(
            lambda prompt, neg_prompt, steps: generate_image_enhanced(prompt, neg_prompt, steps),
            inputs=[prompt_input, negative_prompt, steps_slider],
            outputs=image_out
        )

    with gr.Tab("🔍 Web Search", elem_classes="feature-card"):
        gr.Markdown("### 🌐 Enhanced Google Search with Rich Results")

        with gr.Row():
            with gr.Column(scale=1):
                search_input = gr.Textbox(
                    label="🔍 Search Query",
                    lines=2,
                    placeholder="Enter your search terms..."
                )

                num_results = gr.Slider(3, 10, value=8, step=1, label="Number of Results")
                search_btn = gr.Button("🔍 Search Web", variant="primary", size="lg")

            with gr.Column(scale=2):
                search_out = gr.Textbox(
                    label="🌐 Search Results",
                    lines=15,
                    placeholder="Search results will appear here...",
                    show_copy_button=True
                )

        search_btn.click(
            lambda query, num: search_web_enhanced(query, num),
            inputs=[search_input, num_results],
            outputs=search_out
        )

    # Footer
    gr.Markdown("""
    ---
    ### 🔧 System Status & Enhanced Features
    **Models:** Whisper-Small (OpenVINO) • T5-Base Summarizer (OpenVINO) • Stable Diffusion v1.5 <br>
    **Enhanced Pipeline:** Video → Audio → Transcript → Summary → Contextual Analysis → Focused Images + Targeted Search <br>
    **New Features:** Keyword extraction • Content-aware prompts • Multi-query search • Contextual correlation <br>
    **Optimizations:** Memory-efficient • Progress tracking • Error handling • Enhanced accuracy <br>
    """)

    # Final Credits
    gr.HTML('<div class="credits">🎓 Enhanced AI Classroom Assistant - Made by Kewal Thacker and Siddharth Subramanian</div>')

print("🚀 Launching Enhanced Multimodal AI Classroom Assistant with Unified Pipeline...")
demo.launch(share=True, inbrowser=True)

🚀 Launching Enhanced Multimodal AI Classroom Assistant with Unified Pipeline...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6b5ed6946825050b51.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
